Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [ ]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY = 0.999


In [ ]:
#!wget --no-check-certificate \
#    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
#    -O "F:/coursera/Intro-Tensorflow/images/happy-or-sad.zip"

# Try run the following command in anaconda prompt directly:
# wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip -O F:/coursera/Intro-Tensorflow/images/happy-or-sad.zip
image_dir = 'F:/coursera/TensorflowInPractice/Intro-Tensorflow/images/'
zip_path  = image_dir + 'happy-or-sad.zip'

In [ ]:
print(image_dir)
print(zip_path)

In [ ]:
zip_ref = zipfile.ZipFile(zip_path, 'r')
zip_ref.extractall(image_dir + "h-or-s")
zip_ref.close()


Check the contents of the image directory

In [ ]:
happy_dir = os.path.join(image_dir,'h-or-s/happy')
sad_dir   = os.path.join(image_dir,'h-or-s/sad')

h_images = os.listdir(happy_dir)
print(h_images[:10])

s_images = os.listdir(sad_dir)
print(s_images[:10])

In [ ]:
print('total training happy images:', len(os.listdir(happy_dir)))
print('total training sad images:', len(os.listdir(sad_dir)))

Now let's take a look at a few pictures to get a better sense of what they look like. First, configure the matplot parameters:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

one_image = plt.imread(os.path.join(happy_dir, h_images[0]))
print(one_image.shape)
plt.subplot(1, 4, 1)
plt.imshow(one_image[:,:,0])
plt.subplot(1, 4, 2)
plt.imshow(one_image[:,:,1])
plt.subplot(1, 4, 3)
plt.imshow(one_image[:,:,2])
plt.subplot(1, 4, 4)
plt.imshow(one_image[:,:,3])

plt.figure()
plt.imshow(one_image)

Why are there 4 channels in the image?
But in the CNN model, set channel number to 4 in the input shape will cause error. WHY???!!!

In [ ]:
# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

Now, display a batch of 8 horse and 8 human pictures. You can rerun the cell to see a fresh batch each time:

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 3, nrows * 3)

pic_index += 8
next_happy_pix = [os.path.join(happy_dir, fname) 
                for fname in h_images[pic_index-8:pic_index]]
next_sad_pix = [os.path.join(sad_dir, fname) 
                for fname in s_images[pic_index-8:pic_index]]

for i, img_path in enumerate(next_happy_pix+next_sad_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)
  
plt.show()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>DESIRED_ACCURACY):
      print("\nReached desired accuracy so cancelling training!")
      self.model.stop_training = True    
    
callbacks = myCallback()


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])
model.summary()

In [ ]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)
image_path = image_dir +  'h-or-s'

train_generator = train_datagen.flow_from_directory(
        image_path,                        # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to img_size*img_size
        batch_size =10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode ='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

In [ ]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
      train_generator,
      steps_per_epoch=2,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])
          
# Expected output: "Reached 99.9% accuracy so cancelling training!""

2019-12-6 chenxy  
 It seems that the training performance is not so stable.